# Load data

In [1]:
from keras.datasets import mnist
from PIL import Image
import numpy as np
from keras.utils import to_categorical
from sklearn.utils import shuffle
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras import optimizers
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# --------------------- processing data ----------------------
# loading data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_classes = 10

# form one hot label
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

def resize_image(data):
    img = Image.fromarray(data.reshape(28,28))
    img = img.resize(size=(32,32))
    return np.array(img)

# preprocessing: resize to 32,32. scale to [0,1]
X_train = np.array([resize_image(x) for x in X_train])
X_test = np.array([resize_image(x) for x in X_test])




In [3]:
def rotation_imgs(X, degree):
    size = 32,32
    ans = []
    for x in X:
        img = Image.fromarray(x.reshape(size), mode='L')
        #print(img.mode)
        img.rotate(degree)
        ans.append(np.array(img))
        #print(np.array(img).shape)
    return np.array(ans)

from PIL import ImageFilter

def blur_imgs(X, radius):
    size = 32,32
    ans = []
    for x in X:
        img = Image.fromarray(x.reshape(size), mode='L')
        #print(img.mode)
        img.filter(ImageFilter.GaussianBlur(radius=radius))
        ans.append(np.array(img))
        #print(np.array(img).shape)
    return np.array(ans)
        
        
        

# Build NN

In [15]:
# ------------------- create model -------------------
img_rows, img_cols = 32, 32
def build_model(regulaizer=None):
    model = Sequential()
    # add model layers


    # 32, 32
    channel_multiply = 4

    model.add(Conv2D(1 * channel_multiply, (3, 3), activation='relu', padding='same',
                     input_shape=input_shape, kernel_regularizer=regulaizer))
    # 32, 32
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    # 16, 16
    model.add(Conv2D(2 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 16, 16
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    # 8, 8
    model.add(Conv2D(4 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 8, 8
    model.add(Conv2D(4 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 8, 8
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    # 4, 4
    model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 4, 4
    model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 4, 4
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    # 2, 2
    model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 2, 2
    model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regulaizer))
    # 2, 2
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    # 1, 1
    model.add(Flatten())
    model.add(Dense(4 * channel_multiply, kernel_regularizer=regulaizer))
    model.add(Dense(4 * channel_multiply, kernel_regularizer=regulaizer))
    # model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return(model)

In [5]:
# plot
def plot_img(Y1, Y2, title):
    plt.clf()
    plt.plot(Y1)
    plt.plot(Y2)
    plt.title(title)
    plt.ylabel(title)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(title + '.png')



In [6]:
X_train, y_train = shuffle(X_train, y_train, random_state=13)

In [21]:
def train(model,rotation=0, blur=0, epoches=5, name=""):
    sgd = optimizers.SGD(lr=1e-8, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adamax(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-5)
    ada = optimizers.Adadelta()
    n = X_train.shape[0]
    #print(X_train.shape)
    #print(X_test.shape)
    m = X_test.shape[0]
    X_train_ = X_train.reshape(n, 32,32,1).astype('float32')
    X_test_ = X_test
    if rotation:
        X_test_ = rotation_imgs(X_test_, rotation)
    if blur:
        X_test_ = blur_imgs(X_test_, blur)
    X_test_ = X_test_.reshape(m, 32,32,1).astype('float32')
    
    X_train_ /= 255.0
    X_test_ /= 255.0

    model.compile(optimizer=adam, loss=keras.losses.categorical_crossentropy, 
                  metrics=['accuracy'])
    tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,
                                         write_graph=True, write_images=True)
    history = model.fit(X_train_, y_train, validation_data=(X_test_, y_test), 
                        epochs=epoches, batch_size=32, callbacks=[tbCallBack], verbose=False)
    plot_img(history.history['acc'], history.history['val_acc'], 
             name+"rotation_"+str(rotation) + ", blur_" + str(blur)+" Accuracy")
    plot_img(history.history['loss'], history.history['val_loss'], 
             name+"rotation_"+str(rotation) + ", blur_" + str(blur)+" Loss")
    print(history.history['acc'][-1],'\t',history.history['loss'][-1],'\t',
          history.history['val_acc'][-1],'\t',history.history['val_loss'][-1])



In [ ]:
for rotation in range(-45,45,5):
    train(build_model(),rotation, 0, 3)
for blur in range(7):
    train(build_model(),0, blur, 3)

# regularizer

In [ ]:
for rotation in range(-45,45,5):
    train(build_model(keras.regularizers.l2(0.01)), rotation, 0, 3, 'L2_')
for blur in range(7):
    train(build_model(keras.regularizers.l2(0.01)), 0, blur, 3, "L2_")

In [ ]:
# data augmentation